# Data Cleaning
## Property Market

In [1]:
# General Data Analysis
import pandas as pd
import numpy as np

# Text Based Libraries
import re
import nltk
import gensim
from nltk.stem import SnowballStemmer   
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.parsing.preprocessing import STOPWORDS
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# Import Created Custom Functions from other files
from ipynb.fs.full.TextFunctions import abbr_or_lower, tokenize, find_words, get_feature

In [3]:
# Load Scraped Data
prop_market = pd.read_csv('prop_market.csv')
print('Property Market Dataframe Size :', prop_market.shape)
prop_market.head()

Property Market Dataframe Size : (5608, 9)


,Links,Locality,Price,Beds,Property Type,Baths,Size,Description,Features
0,https://www.propertymarket.com.mt/view/2-bedro...,Attard,349000,2,Town House,2.0,105.0,Property descriptionBIRKIRKARA – Located in a ...,NaN
1,https://www.propertymarket.com.mt/view/2-bedro...,Marsascala,300000,2,Apartment,2.0,107.0,Property descriptionA 2 Bedroom second floor A...,"['Double Glazing', 'Dining room', 'Kitchen', '..."
2,https://www.propertymarket.com.mt/view/2-bedro...,Birkirkara,247000,2,Apartment,NaN,100.0,Property descriptionModern spacious 2 bedroom ...,"['Balcony', 'Finished', 'Freehold', 'Furnished..."
3,https://www.propertymarket.com.mt/view/2-bedro...,San Gwann,330000,2,Maisonette,2.0,133.0,Property descriptionSituated on the outskirts ...,"['Freehold', 'Garage optional (two car)']"
4,https://www.propertymarket.com.mt/view/3-bedro...,Balzan,425000,3,Apartment,2.0,156.0,Property descriptionA very well located modern...,"['Balcony', 'Bathroom', 'Bedroom', 'Dining roo..."


In [4]:
# Quick Check of Nulls
prop_market.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5608 entries, 0 to 5607
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Links          5608 non-null   object 
 1   Locality       5608 non-null   object 
 2   Price          5608 non-null   object 
 3   Beds           5608 non-null   int64  
 4   Property Type  5608 non-null   object 
 5   Baths          5467 non-null   float64
 6   Size           2536 non-null   float64
 7   Description    5529 non-null   object 
 8   Features       3659 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 394.4+ KB


In [5]:
# Drop Rows with Nulls in Baths and Description Columns
prop_market = prop_market.dropna(subset = ['Baths', 'Description'])

# Re-Check Status of Nulls
prop_market.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5391 entries, 0 to 5607
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Links          5391 non-null   object 
 1   Locality       5391 non-null   object 
 2   Price          5391 non-null   object 
 3   Beds           5391 non-null   int64  
 4   Property Type  5391 non-null   object 
 5   Baths          5391 non-null   float64
 6   Size           2450 non-null   float64
 7   Description    5391 non-null   object 
 8   Features       3562 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 421.2+ KB


In [6]:
# Drop Size and Features Column - Excessive Portion of Nulls
prop_market = prop_market.drop(['Size', 'Features'], axis=1)

In [7]:
# Get Baths as Integer
prop_market['Baths'] = prop_market['Baths'].astype(int)

In [8]:
# Convert Price Column Type for better Analysis

# Correct for Price Ranges
prop_market = prop_market.reset_index()
prop_market = prop_market.drop(['index'], axis=1)
for i in range(len(prop_market)):
    if len(prop_market.loc[i, 'Price']) > 8:
        prop_market.loc[i, 'Price'] = (int(re.search('(.*) ', prop_market.loc[i, 'Price']).group(1)) + int(re.search(' (.*)', prop_market.loc[i, 'Price']).group(1)))/2
        
prop_market['Price'] = prop_market['Price'].astype(int)

In [9]:
# Quick check of dataframe
prop_market.describe(include = ['object', 'float', 'int'])

,Links,Locality,Price,Beds,Property Type,Baths,Description
count,5391,5391,5.391000e+03,5391.000000,5391,5391.000000,5391
unique,5391,102,NaN,NaN,26,NaN,5355
top,https://www.propertymarket.com.mt/view/2-bedro...,Mosta,NaN,NaN,Apartment,NaN,Property descriptionA great opportunity has ar...
freq,1,381,NaN,NaN,2795,NaN,3
mean,NaN,NaN,5.203358e+05,2.645149,NaN,2.053793,NaN
std,NaN,NaN,7.169393e+05,0.836346,NaN,0.972574,NaN
min,NaN,NaN,1.000000e+00,1.000000,NaN,1.000000,NaN
25%,NaN,NaN,2.550000e+05,2.000000,NaN,2.000000,NaN
50%,NaN,NaN,3.300000e+05,3.000000,NaN,2.000000,NaN
75%,NaN,NaN,4.950000e+05,3.000000,NaN,2.000000,NaN


Notes:
* Number of unique localities is more than the number of local councils in Malta & Gozo (68)
* Prices of 1 are usually alternatives to writing 'Price on Request' and prices less than 2,000 EUR are assumed incorrect
* Large Variety of Property Types
* Description requires text cleaning

#### Locality

In [10]:
prop_market['Locality'].value_counts()

Mosta                        381
St Paul's Bay                305
Sliema                       261
Birkirkara                   207
Mellieha                     199
                            ... 
Birguma                        1
Mensija                        1
Gozo - Mgarr                   1
The Gardens (St Julian's)      1
Fleur-de-Lys                   1
Name: Locality, Length: 102, dtype: int64

Low observation counts for hamlets and high-rise developments. This can lead to statitical insignificance in modelling.

In [11]:
# Show included localities
prop_market['Locality'].unique()

array(['Attard', 'Marsascala', 'San Gwann', 'Balzan', 'Zebbug', 'Sliema',
       'Naxxar', 'Rabat', 'Swieqi', 'Birkirkara', 'Marsaxlokk', 'Bahrija',
       'Tarxien', 'Zurrieq', 'Mosta', 'Qormi', 'Birzebbuga', 'Portomaso',
       'Vittoriosa (Birgu)', "St Paul's Bay", 'Gozo - Victoria', 'Fgura',
       'Mgarr', "St Julian's", 'Pendergardens', 'Gozo - Ghajnsielem',
       'Cospicua (Bormla)', 'Kalkara', 'Hamrun', 'Gozo - Sannat',
       'Mellieha', 'Santa Venera', 'Paola', 'Zabbar', 'Maghtab', 'Pieta',
       'Guardamangia', 'Santa Maria Estate (Mellieha)', 'Bugibba',
       'Iklin', 'Madliena', 'Gozo - Gharb', 'Msida',
       'Gozo - Santa Lucija', 'Xemxija', 'Bahar Ic-Caghaq', "Ta' Xbiex",
       'Birguma', 'Gozo - Qala', 'Valletta', 'Qawra', 'Zejtun',
       "Ta' l-Ibragg", 'Marsa', 'Siggiewi', 'Gozo - Zebbug', 'Mqabba',
       'Gharghur', 'Gozo - Xaghra', 'Gzira', 'Gudja', 'Ghaxaq', 'Luqa',
       'Gozo - Xlendi', 'Safi', 'Burmarrad', 'Senglea', 'Gozo - Fontana',
       'San Pawl ta

In [12]:
# Assign Hamlets and Highrises to respective Local Councils

prop_market['Locality'] = np.where((prop_market['Locality'] == 'Guardamangia'), 'Pieta', prop_market['Locality'])

prop_market['Locality'] = np.where((prop_market['Locality'] == 'Zebbiegh'), 'Mgarr', prop_market['Locality'])

prop_market['Locality'] = np.where((prop_market['Locality'] == 'Gozo - Santa Lucija'), 'Gozo - Kercem', prop_market['Locality'])

prop_market['Locality'] = np.where((prop_market['Locality'] == 'Gozo - Marsalforn'), 'Gozo - Zebbug', prop_market['Locality'])

prop_market['Locality'] = np.where((prop_market['Locality'] == 'Gozo - Mgarr'), 'Gozo - Ghajnsielem', prop_market['Locality'])

prop_market['Locality'] = np.where((prop_market['Locality'] == 'Gozo - Xlendi'), 'Gozo - Munxar', prop_market['Locality'])

sgn = ['Kappara', 'Mensija']
for i in sgn:
    prop_market['Locality'] = np.where((prop_market['Locality'] == i), 'San Gwann', prop_market['Locality'])

bkr = ['Swatar', 'Fleur-de-Lys']
for i in bkr:
    prop_market['Locality'] = np.where((prop_market['Locality'] == i), 'Birkirkara', prop_market['Locality'])

slm = ['Tigne Point', 'Fort Cambridge']
for i in slm:
    prop_market['Locality'] = np.where((prop_market['Locality'] == i), 'Sliema', prop_market['Locality'])

rbt = ['Bahrija', 'Bingemma']
for i in rbt:
    prop_market['Locality'] = np.where((prop_market['Locality'] == i), 'Rabat', prop_market['Locality'])

stj = ['Portomaso', 'Pendergardens', 'Paceville', "Ta' Giorni", "The Gardens (St Julian's)"]
for i in stj:
    prop_market['Locality'] = np.where((prop_market['Locality'] == i), "St Julian's", prop_market['Locality'])
    
nxr = ['Maghtab', 'Bahar Ic-Caghaq', 'Birguma', 'San Pawl tat Targa', 'Salina']
for i in nxr:
    prop_market['Locality'] = np.where((prop_market['Locality'] == i), 'Naxxar', prop_market['Locality'])
    
mlh = ['Santa Maria Estate (Mellieha)', 'Manikata']
for i in mlh:
    prop_market['Locality'] = np.where((prop_market['Locality'] == i), 'Mellieha', prop_market['Locality'])
                                       
spb = ['Bugibba', 'Xemxija', 'Qawra', 'Burmarrad', 'Wardija']
for i in spb:
    prop_market['Locality'] = np.where((prop_market['Locality'] == i), "St Paul's Bay", prop_market['Locality'])
    
swq = ['Madliena', "Ta' l-Ibragg", 'High Ridge']
for i in swq:
    prop_market['Locality'] = np.where((prop_market['Locality'] == i), 'Swieqi', prop_market['Locality'])
    
print('Number of Localities :', len(prop_market['Locality'].unique().tolist()))

Number of Localities : 68


#### Price

In [13]:
# Remove Prices below 2000 
prop_market = prop_market[prop_market['Price'] > 2000]
print('Dataframe Size :', prop_market.shape)

Dataframe Size : (5385, 7)


#### Property Types

In [14]:
# Show included property types
prop_market['Property Type'].unique()

array(['Town House', 'Apartment', 'Maisonette', 'Penthouse', 'Palazzo',
       'House of Character', 'Terraced House', 'Bungalow', 'Villa',
       'House', 'Semi-Detached House', 'Duplex', 'Semi-Detached Bungalow',
       'Farm House', 'Semi-Detached Villa', 'Detached Villa',
       'Village House', 'Site', 'Apartment For Sale in',
       'Penthouse For Sale in', 'Hotel', 'Detached Bungalow', 'Plot',
       'Guest House', 'Terraced House For Sale in', 'Block of Apartments'],
      dtype=object)

In [15]:
# Adjust Scraping Errors

prop_market['Property Type'] = np.where((prop_market['Property Type'] == 'Apartment For Sale in'), 'Apartment', prop_market['Property Type'])

prop_market['Property Type'] = np.where((prop_market['Property Type'] == 'Penthouse For Sale in'), 'Penthouse', prop_market['Property Type'])

prop_market['Property Type'] = np.where((prop_market['Property Type'] == 'Terraced House For Sale in'), 'Terraced House', prop_market['Property Type'])

#### Description

In [16]:
# Remove Common Start of Description Text
prop_market['Description'] = prop_market['Description'].str.replace('Property description', '')

In [17]:
# Clean Text with lowercase and lemmatization
prop_market['Description - Clean'] = [tokenize(listing, 2) for listing in prop_market['Description']]

## Create Features From Text

#### Garden

In [18]:
# Find words relating to gardens
find_words(prop_market['Description - Clean'], 'gard', 'in')

{'backgarden',
 'garden',
 'garden2',
 'gardenfreeholde',
 'gardenget',
 'gardenia',
 'gardening',
 'gardenluxury',
 'gardensexperience',
 'gardenstreet',
 'gardenuca',
 'gardjola',
 'pendergardens',
 'regarded',
 'terracegarden'}

In [19]:
# Get Dummy for Properties with Gardens
garden_words = ['backgarden', 'garden', 'garden2', 'gardenfreeholde', 'gardenget', 'gardenia', 'gardening', 'gardenluxury',
                'gardensexperience', 'gardenstreet', 'gardenuca', 'terracegarden']
prop_market['Garden'] = get_feature(prop_market['Description - Clean'], garden_words)

prop_market['Garden'].value_counts()

0    4919
1     466
Name: Garden, dtype: int64

#### Yard

In [20]:
# Find words relating to yards
find_words(prop_market['Description - Clean'], 'yard', 'in')

{'backyard',
 'backyard1',
 'backyardbox',
 'backyardfreeholde',
 'backyardinternal',
 'backyardlaundry',
 'backyardmain',
 'backyardref',
 'backyardsroof',
 'backyardthis',
 'bathroombackyard',
 'bathroombackyardfreeholde',
 'bathroombackyardlaundry',
 'bathroombackyardmaltese',
 'bathroombackyardoptional',
 'bathroombackyardpoolbalconyterraceroof',
 'bathroombackyardroof',
 'bathroomcourtyardlounge',
 'bathroomcourtyardmassive',
 'bathroomcourtyardroof',
 'bathroomsbackyard',
 'bathroomsbackyardlaundry',
 'bathroomsbackyardroof',
 'bathroomsbalconybackyardoptional',
 'bathroomsterraceyardroof',
 'bathroomstudybackyardfreeholde',
 'bathroomyard',
 'bathroomyardfreeholde',
 'bedroomsbathroomcourtyard',
 'corneryardmarbled',
 'courtyard',
 'courtyard2',
 'courtyard4',
 'courtyardbalconiesroof',
 'courtyardlocated',
 'courtyardroof',
 'courtyardterracefreeholde',
 'diningbackyard',
 'floormaisonettebackyarddesigner',
 'inbackyardfreeholde',
 'roombackyardcall',
 'roombackyardfreeholde',


In [21]:
# Get Dummy for Properties with Yards
yard_words = find_words(prop_market['Description - Clean'], 'yard', 'in')
prop_market['Yard'] = get_feature(prop_market['Description - Clean'], yard_words)

prop_market['Yard'].value_counts()

0    4276
1    1109
Name: Yard, dtype: int64

#### Terrace

In [22]:
# Find words relating to terraces
find_words(prop_market['Description - Clean'], 'terrace', 'in')

{'balconyterraceroof',
 'bathroombackyardpoolbalconyterraceroof',
 'bathroomsterrace',
 'bathroomsterraceyardroof',
 'bathroomterrace',
 'bathroomterracebalconyroof',
 'bathroomterracefreeholde',
 'bathroomterraceroof',
 'courtyardterracefreeholde',
 'diningterraceone',
 'edgeterraced',
 'roomterrace',
 'terrace',
 'terrace2',
 'terraceasking',
 'terraceback',
 'terracebalconyfreeholde',
 'terracebalconyserved',
 'terracecall',
 'terraced',
 'terracefinished',
 'terracefreeholde',
 'terracegarden',
 'terracehighly',
 'terraceincluding',
 'terraceinternal',
 'terraceoptional',
 'terraceref',
 'terraceroof',
 'terraces3',
 'terracesecond',
 'terraceserved',
 'terracesfreeholde',
 'terracestep',
 'terracesthis',
 'terracesupvc',
 'terracethis',
 'thisterraced'}

In [23]:
# Get Dummy for Properties with Terraces
terrace_words = ['balconyterraceroof', 'bathroombackyardpoolbalconyterraceroof', 'bathroomsterrace', 
              'bathroomsterraceyardroof', 'bathroomterrace', 'bathroomterracebalconyroof', 'bathroomterracefreeholde', 
              'bathroomterraceroof', 'courtyardterracefreeholde', 'diningterraceone', 'edgeterraced', 'roomterrace',
              'terrace', 'terrace2', 'terraceasking', 'terraceback', 'terracebalconyfreeholde', 'terracebalconyserved', 
              'terracecall', 'terracefinished', 'terracefreeholde', 'terracegarden', 'terracehighly', 'terraceincluding',
              'terraceinternal', 'terraceoptional', 'terraceref', 'terraceroof', 'terraces3', 'terracesecond',
              'terraceserved', 'terracesfreeholde', 'terracestep', 'terracesthis', 'terracesupvc', 'terracethis']
prop_market['Terrace'] = get_feature(prop_market['Description - Clean'], terrace_words)

prop_market['Terrace'].value_counts()

0    2852
1    2533
Name: Terrace, dtype: int64

#### Lift

In [24]:
# Find words relating to lifts
find_words(prop_market['Description - Clean'], 'lift', 'in')

{'garagelift',
 'lift',
 'liftairspace',
 'liftfreehold',
 'liftfreeholde',
 'liftown',
 'liftpartly',
 'liftref',
 'liftroof'}

In [25]:
# Get Dummy for Properties with Lifts
lift_words = ['garagelift', 'lift', 'liftairspace', 'liftfreehold', 'liftfreeholde', 'liftown', 'liftpartly', 'liftref',
              'liftroof']
prop_market['Lift'] = get_feature(prop_market['Description - Clean'], lift_words)

prop_market['Lift'].value_counts()

0    4350
1    1035
Name: Lift, dtype: int64

#### Pool

In [26]:
# Find words relating to pools
find_words(prop_market['Description - Clean'], 'pool', 'in')

{'60sqmpool',
 'bathroombackyardpoolbalconyterraceroof',
 'bathroompool',
 'pool',
 'pool1',
 'poolfreeholde',
 'poolget',
 'poolref',
 'poolside',
 'poolsref',
 'pooluca',
 'whirlpool',
 'whirpool'}

In [27]:
# Get Dummy for Properties with Pools
pool_words = ['60sqmpool', 'bathroombackyardpoolbalconyterraceroof', 'bathroompool', 'pool', 'pool1', 'poolfreeholde',
                'poolget', 'poolref', 'poolside', 'poolsref', 'pooluca']
prop_market['Pool'] = get_feature(prop_market['Description - Clean'], pool_words)

prop_market['Pool'].value_counts()

0    4622
1     763
Name: Pool, dtype: int64

#### Roof Access

In [28]:
# Find words relating to roofs
find_words(prop_market['Description - Clean'], 'roof', 'in')

{'130sqmroof',
 'areabalconyroof',
 'backyardsroof',
 'balconyroof',
 'balconyterraceroof',
 'bathroombackyardpoolbalconyterraceroof',
 'bathroombackyardroof',
 'bathroombalconygarageroof',
 'bathroombalconyroof',
 'bathroomcourtyardroof',
 'bathroomroof',
 'bathroomsbackyardroof',
 'bathroomsbalconyroof',
 'bathroomsroof',
 'bathroomsterraceyardroof',
 'bathroomterracebalconyroof',
 'bathroomterraceroof',
 'bedroomsroof',
 'courtyardbalconiesroof',
 'courtyardroof',
 'floorwashroomroof',
 'garageroof',
 'liftroof',
 'maisonetteroof',
 'proof',
 'proofed',
 'proofing',
 'roof',
 'roofed',
 'rooffreeholde',
 'rooffreeholdoptional',
 'roofonly',
 'roofprivate',
 'roofref',
 'roofsituated',
 'roofthis',
 'rooftop',
 'roofucafreehold79026090ref',
 'roomroof',
 'soundproofing',
 'terraceroof',
 'toproof',
 'wardroberoof',
 'washroomroof',
 'waterproofed',
 'waterproofing',
 'yardbalconyroof',
 'yardroof'}

In [29]:
# Get Dummy for Properties with Roof Access
roof_words = ['130sqmroof', 'areabalconyroof', 'backyardsroof', 'balconyroof', 'balconyterraceroof',
              'bathroombackyardpoolbalconyterraceroof', 'bathroombackyardroof', 'bathroombalconygarageroof',
              'bathroombalconyroof', 'bathroomcourtyardroof', 'bathroomroof', 'bathroomsbackyardroof',
              'bathroomsbalconyroof', 'bathroomsroof', 'bathroomsterraceyardroof', 'bathroomterracebalconyroof',
              'bathroomterraceroof', 'bedroomsroof', 'courtyardbalconiesroof', 'courtyardroof', 'floorwashroomroof',
              'garageroof', 'liftroof', 'maisonetteroof', 'roof', 'rooffreeholde', 'rooffreeholdoptional', 'roofonly',
              'roofprivate', 'roofref', 'roofsituated', 'roofthis', 'rooftop', 'roofucafreehold79026090ref', 'roomroof',
              'terraceroof', 'toproof', 'wardroberoof', 'washroomroof', 'yardbalconyroof', 'yardroof']
prop_market['Roof Access'] = get_feature(prop_market['Description - Clean'], roof_words)

prop_market['Roof Access'].value_counts()

0    4381
1    1004
Name: Roof Access, dtype: int64

#### Seafront

In [30]:
# Find words relating to seafront properties
find_words(prop_market['Description - Clean'], 'sea', 'in')

{'brendaseafront',
 'cliffsearly',
 'sea',
 'seafront',
 'seafrontref',
 'seafrontstructurally',
 'seainternal',
 'seamless',
 'seamlessly',
 'search',
 'seashore',
 'seaside',
 'season',
 'seasonal',
 'seater',
 'seating',
 'seaview',
 'seaview3',
 'seaviews',
 'seaviewsopen',
 'seaviewsref',
 'standardssea'}

In [31]:
# Get Dummy for Properties at the Seafront
sea_words = ['brendaseafront', 'seafront', 'seafrontref', 'seafrontstructurally', 'seashore', 'seaside']
prop_market['Seafront'] = get_feature(prop_market['Description - Clean'], sea_words)

prop_market['Seafront'].value_counts()

0    5123
1     262
Name: Seafront, dtype: int64

#### Views

In [32]:
# Find words relating to properties with views
find_words(prop_market['Description - Clean'], 'view', 'in')

{'2023viewings',
 'kitchenviewopen',
 'seaview',
 'seaview3',
 'seaviews',
 'seaviewsopen',
 'seaviewsref',
 'view',
 'viewanna',
 'viewavailableprice',
 'viewcentrally',
 'viewed',
 'viewing',
 'viewinganna',
 'viewingnicole',
 'viewingref',
 'viewingrobin',
 'viewingsa',
 'viewingslocated',
 'viewingsnew',
 'viewingsopportunity',
 'viewingsref',
 'viewingsthis',
 'viewluxurious',
 'viewpoint',
 'views1',
 'viewsapartment',
 'viewsasking',
 'viewscall',
 'viewscirca',
 'viewsenjoy',
 'viewsfinished',
 'viewsfully',
 'viewsref'}

In [33]:
# Get Dummy for Properties with Views
view_words = ['kitchenviewopen', 'seaview', 'seaview3', 'seaviews', 'seaviewsopen', 'seaviewsref', 'views1',
              'viewsapartment', 'viewsasking', 'viewscall', 'viewscirca', 'viewsenjoy', 'viewsfinished', 'viewsfully',
              'viewsref']
prop_market['Views'] = get_feature(prop_market['Description - Clean'], view_words)

prop_market['Views'].value_counts()

0    5354
1      31
Name: Views, dtype: int64

#### Garage

In [34]:
# Find words relating to properties with garages
find_words(prop_market['Description - Clean'], 'gar', 'in')

{'backgarden',
 'bathroombalconygarageroof',
 'cigar',
 'garage',
 'garage315sqm',
 'garageairspacefreeholde',
 'garageasking',
 'garageavailable',
 'garagecall',
 'garagefinished',
 'garagefreeholde',
 'garageget',
 'garageinterconnected',
 'garageit',
 'garagelift',
 'garagelovely',
 'garagepermits',
 'garagepossibility',
 'garageref',
 'garageroof',
 'garageserved',
 'garagesfreeholde',
 'garagesref',
 'garagestudyreading',
 'garagetotal',
 'garageuca',
 'garden',
 'garden2',
 'gardenfreeholde',
 'gardenget',
 'gardenia',
 'gardening',
 'gardenluxury',
 'gardensexperience',
 'gardenstreet',
 'gardenuca',
 'gardjola',
 'garett',
 'garges',
 'garigor',
 'garrigor',
 'imgarr',
 'maisonettesapartmentspenthousesgaragesre',
 'margarita',
 'mgarr',
 'pendergardens',
 'regarded',
 'terracegarden',
 'torrimaisonettesapartmentspenthousesgaragesre'}

In [35]:
# Get Dummy for Properties with Garages
garage_words = ['bathroombalconygarageroof', 'garage', 'garage315sqm', 'garageairspacefreeholde', 'garageasking',
                'garageavailable', 'garagecall', 'garagefinished', 'garagefreeholde', 'garageget', 'garageinterconnected',
                'garageit', 'garagelift', 'garagelovely', 'garagepermits', 'garagepossibility', 'garageref', 'garageroof',
                'garageserved', 'garagesfreeholde', 'garagesref', 'garagestudyreading', 'garagetotal', 'garageuca',
                'garges', 'maisonettesapartmentspenthousesgaragesre', 'torrimaisonettesapartmentspenthousesgaragesre']
prop_market['Garage'] = get_feature(prop_market['Description - Clean'], garage_words)

prop_market['Garage'].value_counts()

0    2816
1    2569
Name: Garage, dtype: int64

#### Garage Option

Using bigrams to see words that denote 'option' relating to gragaes in the descriptions.

In [36]:
# Instanciate TfidfVectorizer (tokenizer set to iterate over documents/paragraphs)
tfidf2 = TfidfVectorizer(ngram_range=(2, 2), tokenizer=lambda doc: doc, lowercase=False) # Bigrams

In [37]:
# Keep descriptions that references garages
garage_in_desc = []
for i in prop_market['Description - Clean']:
    if len(set(i).intersection(garage_words)) != 0:
        garage_in_desc.append(i)

In [38]:
# Bigrams
tfidf_description = tfidf2.fit_transform(garage_in_desc)
bigrams_df = pd.DataFrame.sparse.from_spmatrix(tfidf_description, columns=tfidf2.get_feature_names_out())
desc_bigrams = list(bigrams_df.columns)

In [39]:
# Get bigrams that reference garages
garage_bigrams = []
for i in desc_bigrams:
    if 'garage' in i:
        garage_bigrams.append(i)
        
print('Number of Bigrams Referencing Garages :', len(garage_bigrams))

Number of Bigrams Referencing Garages : 516


In [40]:
# Manual identification of bigrams with references to garages on offer
garage_opt_bi = ['2023optional garage', '2024optional garage', '2025optional garage', '235koptional garage',
                 '30sqmoptional garagefreeholde', 'availability garage', 'available garage',
                 'bathroomsbalconybackyardoptional garageroof', 'buy garage', 'car garagepossibility',
                 'conditionedoptional garage', 'freeholdoptional garage', 'garage additional', 'garage avaialble',
                 'garage availabel', 'garage available', 'garage availablecontact', 'garage availableextras',
                 'garage availableget', 'garage availableproject', 'garage availableref', 'garage avilable',
                 'garage offer', 'garage offered', 'garage offering', 'garage opt', 'garage option', 'garage optional',
                 'garage optionalref', 'garage possibility', 'garage possible', 'garage possibly', 'garage potential',
                 'garage rent', 'garage rented', 'garage request', 'garage requested', 'garage315sqm asking',
                 'garageasking price', 'garageavailable asking', 'garagepossibility gym38', 'house garageasking',
                 'leasing garage', 'let garage', 'option garage', 'optional garage', 'optional garageavailable',
                 'optional garagefinished', 'optional garagesref', 'purchase garage', 'rent garage', 'rental garage',
                 'rented garage', 'streetoptional garage', 'terraceoptional garage']

In [41]:
# Words Denoting Options
opt_words = ['2023optional', '2024optional', '2025optional', '235koptional', '30sqmoptional', 'additional', 'asking',
             'avaialble', 'availabel', 'availability', 'available', 'availablecontact', 'availableextras', 'availableget',
             'availableproject', 'availableref', 'avilable', 'bathroomsbalconybackyardoptional', 'buy', 'rental', 'rented',
             'conditionedoptional', 'freeholdoptional', 'garageasking', 'garageavailable', 'garagepossibility', 'leasing',
             'let', 'offer', 'offered', 'offering', 'opt', 'option', 'optional', 'optionalref', 'possibility', 'possible',
             'possibly', 'potential', 'purchase', 'rent', 'request', 'requested', 'streetoptional', 'terraceoptional']

In [42]:
# Get dummy for optional garage
garage_opt_dummy = []
for i in prop_market['Description - Clean']:
    # Loop through features to check for 'garage' reference
    if len(set(i).intersection(garage_words)) == 0:
        garage_opt_dummy.append(0)
    else:
        # Check for reference of garage as option
        if len(set(i).intersection(opt_words)) == 0:
            garage_opt_dummy.append(0)
        else:
            garage_opt_dummy.append(1)

# Assign Dummy to Dataframe
prop_market['Optional Garage'] = garage_opt_dummy

prop_market['Optional Garage'].value_counts()

0    3235
1    2150
Name: Optional Garage, dtype: int64

In [43]:
# Correct overlap between dummy for garage and optional garage
prop_market['Garage'] = np.where((prop_market['Optional Garage'] == 1), 0, prop_market['Garage'])
prop_market['Garage'].value_counts() #confirmed, correction worked

0    4966
1     419
Name: Garage, dtype: int64

#### Airspace

In [44]:
# Find words relating to properties with airspace included
find_words(prop_market['Description - Clean'], 'airspace', 'in')

{'airspace',
 'airspace1',
 'airspaceasking',
 'airspaceawesome',
 'airspacebalconyfreeholde',
 'airspacecirca',
 'airspacefreeholde',
 'airspaceget',
 'airspaceideally',
 'airspacelaundry',
 'airspaceoptional',
 'airspaceoutdoor',
 'airspaceprice',
 'airspaceref',
 'airspaceroom',
 'airspacespacious',
 'airspacethis',
 'airspacewellfreeholde',
 'garageairspacefreeholde',
 'liftairspace'}

In [45]:
# Get Dummy for Properties with Airspace
airspace_words = find_words(prop_market['Description - Clean'], 'airspace', 'in')
prop_market['Airspace'] = get_feature(prop_market['Description - Clean'], airspace_words)

prop_market['Airspace'].value_counts()

0    4662
1     723
Name: Airspace, dtype: int64

#### Airspace Option

Words denoting options are assumed to have a common thread between referencing garages and airspace.

In [46]:
# Get dummy for optional airspace
airspace_opt_dummy = []
for i in prop_market['Description - Clean']:
    # Loop through features to check for 'garage' reference
    if len(set(i).intersection(airspace_words)) == 0:
        airspace_opt_dummy.append(0)
    else:
        # Check for reference of garage as option
        if len(set(i).intersection(opt_words)) == 0:
            airspace_opt_dummy.append(0)
        else:
            airspace_opt_dummy.append(1)

# Assign Dummy to Dataframe
prop_market['Optional Airspace'] = airspace_opt_dummy

prop_market['Optional Airspace'].value_counts()

0    4960
1     425
Name: Optional Airspace, dtype: int64

In [47]:
# Correct overlap between dummy for airspace and optional airspace
prop_market['Airspace'] = np.where((prop_market['Optional Airspace'] == 1), 0, prop_market['Airspace'])
prop_market['Airspace'].value_counts() #confirmed, correction worked

0    5087
1     298
Name: Airspace, dtype: int64

#### Furnished

In [48]:
# Find words relating to furnished properties
find_words(prop_market['Description - Clean'], 'furnished', 'in')

{'100sqmfurnished',
 'doorsunfurnishedtotal',
 'furnished',
 'furnished1',
 'furnishedcall',
 'furnishedcontact',
 'furnishedoptional',
 'furnishedref',
 'furnishedsenglea',
 'furnishedserved',
 'refurnished',
 'unfurnished'}

In [49]:
# Get Dummy for Furnished Properties
furnished_words = ['100sqmfurnished', 'doorsunfurnishedtotal', 'furnished', 'furnished1', 'furnishedcall',
                   'furnishedcontact', 'furnishedoptional', 'furnishedref', 'furnishedsenglea', 'furnishedserved',
                   'refurnished']
prop_market['Furnished'] = get_feature(prop_market['Description - Clean'], furnished_words)

prop_market['Furnished'].value_counts()

0    4326
1    1059
Name: Furnished, dtype: int64

#### Finished

In [50]:
# Find words relating to finished properties
find_words(prop_market['Description - Clean'], 'finished', 'in')

{'223kfinished',
 'balconyfinished',
 'finished',
 'finishedoptional',
 'garagefinished',
 'infinished',
 'terracefinished',
 'viewsfinished'}

In [51]:
# Get Dummy for Finished Properties
finished_words = find_words(prop_market['Description - Clean'], 'finished', 'in')
prop_market['Finished'] = get_feature(prop_market['Description - Clean'], finished_words)

prop_market['Finished'].value_counts()

0    2833
1    2552
Name: Finished, dtype: int64

In [52]:
# Correct for crossovers between furnished and finished properties
prop_market['Finished'] = np.where((prop_market['Furnished'] == 1), 0, prop_market['Finished'])
prop_market['Finished'].value_counts() #confirmed, correction worked

0    3104
1    2281
Name: Finished, dtype: int64

<b>NOTE:</b> 3,340 properties are either furnished or finished, the remaining properties are assumed to be sold as shell. Dummy not created for shell properties given that 1 of the referenced features would have to be dropped anyway when modelling to avoid the dummy variable trap.

## Save Final Dataframe

In [53]:
# Remove Cleaned Text Column
prop_market = prop_market.drop(['Description - Clean'], axis=1)

In [54]:
print('Dataframe Size :', prop_market.shape)
prop_market.head()

Dataframe Size : (5385, 21)


,Links,Locality,Price,Beds,Property Type,Baths,Description,Garden,Yard,Terrace,...,Pool,Roof Access,Seafront,Views,Garage,Optional Garage,Airspace,Optional Airspace,Furnished,Finished
0,https://www.propertymarket.com.mt/view/2-bedro...,Attard,349000,2,Town House,2,"BIRKIRKARA – Located in a UCA area, yet in a q...",0,0,1,...,0,1,0,0,0,0,1,0,1,0
1,https://www.propertymarket.com.mt/view/2-bedro...,Marsascala,300000,2,Apartment,2,"A 2 Bedroom second floor Apartment, with permi...",0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,https://www.propertymarket.com.mt/view/2-bedro...,San Gwann,330000,2,Maisonette,2,Situated on the outskirts of San Gwann and few...,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,https://www.propertymarket.com.mt/view/3-bedro...,Balzan,425000,3,Apartment,2,A very well located modern three bedroom 180 S...,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,https://www.propertymarket.com.mt/view/2-bedro...,Zebbug,370000,2,Town House,2,A two bedroom townhouse just minutes walk away...,0,1,0,...,0,1,0,0,0,0,1,0,0,0


In [55]:
# Save Data
prop_market.to_csv('prop_market.csv', index=False)